# Creating your own dataset

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 4.4 MB 66.7 MB/s 
     |████████████████████████████████| 140 kB 79.5 MB/s 
     |████████████████████████████████| 212 kB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.5 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 127 kB 66.6 MB/s 
     |████████████████████████████████| 271 kB 66.0 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 144 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 34.2 MB/s 
     |████████████████████████████████| 1.2 MB 15.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "test@gmail.com"
!git config --global user.name "test"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [4]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?"
response = requests.get(url)

In [6]:
response.status_code

200

In [16]:
response.json()

[{'active_lock_reason': None,
  'assignee': {'avatar_url': 'https://avatars.githubusercontent.com/u/1676121?v=4',
   'events_url': 'https://api.github.com/users/severo/events{/privacy}',
   'followers_url': 'https://api.github.com/users/severo/followers',
   'following_url': 'https://api.github.com/users/severo/following{/other_user}',
   'gists_url': 'https://api.github.com/users/severo/gists{/gist_id}',
   'gravatar_id': '',
   'html_url': 'https://github.com/severo',
   'id': 1676121,
   'login': 'severo',
   'node_id': 'MDQ6VXNlcjE2NzYxMjE=',
   'organizations_url': 'https://api.github.com/users/severo/orgs',
   'received_events_url': 'https://api.github.com/users/severo/received_events',
   'repos_url': 'https://api.github.com/users/severo/repos',
   'site_admin': False,
   'starred_url': 'https://api.github.com/users/severo/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/severo/subscriptions',
   'type': 'User',
   'url': 'https://api.github.com/use

In [ ]:
GITHUB_TOKEN = xxx  # Copy your GitHub token here
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [20]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [21]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [22]:
from datasets import load_dataset
issues_dataset = load_dataset("json", data_files="datasets-issues.jsonl", split="train")
issues_dataset

Using custom data configuration default-b77f9a1497e54911


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b77f9a1497e54911/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 4560
})

In [23]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/issues/436
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/pull/2064
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/2064', 'html_url': 'https://github.com/huggingface/datasets/pull/2064', 'diff_url': 'https://github.com/huggingface/datasets/pull/2064.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/2064.patch', 'merged_at': datetime.datetime(2021, 3, 16, 18, 0, 7)}

>> URL: https://github.com/huggingface/datasets/issues/2489
>> Pull request: None



In [24]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

  0%|          | 0/4560 [00:00<?, ?ex/s]

In [25]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'author_association': 'CONTRIBUTOR',
  'body': "@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
  'created_at': '2021-08-12T12:21:52Z',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'id': 897594128,
  'issue_url': 'https://api.github.com/repos/huggingface/datasets

In [26]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [27]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

  0%|          | 0/4560 [00:00<?, ?ex/s]

In [28]:
issues_with_comments_dataset.to_json("issues-datasets-with-comments.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

20501373

In [29]:
from huggingface_hub import list_datasets

all_datasets = list_datasets()
print(f"Number of datasets on Hub: {len(all_datasets)}")
print(all_datasets[0])

Number of datasets on Hub: 5805
Dataset Name: acronym_identification, Tags: ['arxiv:2010.14678', 'annotations_creators:expert-generated', 'language_creators:found', 'languages:en', 'licenses:mit', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'task_categories:token-classification', 'task_ids:token-classification-other-acronym-identification', 'pretty_name:Acronym Identification Dataset']


In [30]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [31]:
from huggingface_hub import create_repo

repo_url = create_repo(name="github-issues", repo_type="dataset")
repo_url

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:82: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  FutureWarning,


'https://huggingface.co/datasets/XGBooster/github-issues'

In [37]:
from huggingface_hub import Repository

repo = Repository(local_dir="github-issues", clone_from=repo_url)
!cp issues-datasets-with-comments.jsonl github-issues/

/content/github-issues is already a clone of https://huggingface.co/datasets/XGBooster/github-issues. Make sure you pull the latest changes with `repo.git_pull()`.


In [38]:
repo.lfs_track("*.jsonl")

In [39]:
repo.push_to_hub()

Upload file issues-datasets-with-comments.jsonl:   0%|          | 3.34k/19.6M [00:00<?, ?B/s]

To https://huggingface.co/datasets/XGBooster/github-issues
   fb84845..4d91f73  main -> main



'https://huggingface.co/datasets/XGBooster/github-issues/commit/4d91f733adb400420a80ce86b57c1a07f2d8813a'

In [40]:
remote_dataset = load_dataset("XGBooster/github-issues", split="train")
remote_dataset

Using custom data configuration XGBooster--github-issues-e6a561037c64b3a4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/XGBooster___json/XGBooster--github-issues-e6a561037c64b3a4/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request', 'is_pull_request'],
    num_rows: 4560
})